[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/khetansarvesh/CV/blob/main/im_segmentation/UNet.ipynb)

In [1]:
import torch
from torch import optim, nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

import os
from PIL import Image
from torch.utils.data.dataset import Dataset
from torchvision import transforms

import matplotlib.pyplot as plt

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


# **Dataset**
download and upload the train.zip and train_masks.zip file from kaggle [link](https://www.kaggle.com/competitions/carvana-image-masking-challenge/data) here

In [ ]:
# upzipping the two files
!unzip train_masks.zip
!unzip train.zip

In [19]:
# loading dataset
class CarvanaDataset(Dataset):
    def __init__(self, test=False):
        self.images = sorted(["train/"+i for i in os.listdir("train/")])
        self.masks = sorted(["train_masks/"+i for i in os.listdir("train_masks/")])

        self.transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor()])

    def __getitem__(self, index):
        img = Image.open(self.images[index]).convert("RGB")
        mask = Image.open(self.masks[index]).convert("L")

        return self.transform(img), self.transform(mask)

    def __len__(self):
        return len(self.images)

dataset = CarvanaDataset()

In [20]:
# train and validation split
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(dataset, [0.8, 0.2], generator=generator)
print(train_dataset, val_dataset)

<torch.utils.data.dataset.Subset object at 0x785b7d5e2140> <torch.utils.data.dataset.Subset object at 0x785b7d5e3df0>


In [21]:
# train and validation dataloader
train_dataloader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=True)

# **Modelling**

In [22]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_op = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv_op(x)


class DownSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = DoubleConv(in_channels, out_channels)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        down = self.conv(x)
        p = self.pool(down)

        return down, p


class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels//2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
       x1 = self.up(x1)
       x = torch.cat([x1, x2], 1)
       return self.conv(x)

In [23]:
class UNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.down_convolution_1 = DownSample(in_channels, 64)
        self.down_convolution_2 = DownSample(64, 128)
        self.down_convolution_3 = DownSample(128, 256)
        self.down_convolution_4 = DownSample(256, 512)

        self.bottle_neck = DoubleConv(512, 1024)

        self.up_convolution_1 = UpSample(1024, 512)
        self.up_convolution_2 = UpSample(512, 256)
        self.up_convolution_3 = UpSample(256, 128)
        self.up_convolution_4 = UpSample(128, 64)

        self.out = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=1)

    def forward(self, x):
       down_1, p1 = self.down_convolution_1(x)
       down_2, p2 = self.down_convolution_2(p1)
       down_3, p3 = self.down_convolution_3(p2)
       down_4, p4 = self.down_convolution_4(p3)

       b = self.bottle_neck(p4)

       up_1 = self.up_convolution_1(b, down_4)
       up_2 = self.up_convolution_2(up_1, down_3)
       up_3 = self.up_convolution_3(up_2, down_2)
       up_4 = self.up_convolution_4(up_3, down_1)

       out = self.out(up_4)
       return out

In [24]:
model = UNet(in_channels=3, num_classes=1).to(device)
optimizer = optim.AdamW(model.parameters(), lr=3e-4)
criterion = nn.BCEWithLogitsLoss()

# **Training**

In [ ]:
for epoch in tqdm(range(2)):
    model.train()
    train_running_loss = 0
    for idx, img_mask in enumerate(tqdm(train_dataloader)):
        img = img_mask[0].float().to(device)
        mask = img_mask[1].float().to(device)

        y_pred = model(img)
        optimizer.zero_grad()

        loss = criterion(y_pred, mask)
        train_running_loss += loss.item()

        loss.backward()
        optimizer.step()

    train_loss = train_running_loss / (idx + 1)

    model.eval()
    val_running_loss = 0
    with torch.no_grad():
        for idx, img_mask in enumerate(tqdm(val_dataloader)):
            img = img_mask[0].float().to(device)
            mask = img_mask[1].float().to(device)

            y_pred = model(img)
            loss = criterion(y_pred, mask)

            val_running_loss += loss.item()

        val_loss = val_running_loss / (idx + 1)

    print("-"*30)
    print(f"Train Loss EPOCH {epoch+1}: {train_loss:.4f}")
    print(f"Valid Loss EPOCH {epoch+1}: {val_loss:.4f}")
    print("-"*30)

  0%|          | 0/128 [00:00<?, ?it/s]

In [ ]:
torch.save(model.state_dict(), "unet_model.pth")

# **Inference**

In [ ]:
# loading model architecture
model = UNet(in_channels=3, num_classes=1).to(device)

# loading the model with trained weights
model.load_state_dict(torch.load("./models/unet.pth", map_location=torch.device(device)))

# loading testing dataset
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()])

img = transform(Image.open("./data/manual_test/03a857ce842d_15.jpg")).float().to(device)
img = img.unsqueeze(0)


# performing prediction
pred_mask = model(img)

In [ ]:
# visualizing
fig = plt.figure()

# showing the original image after doing some post processing
img = img.squeeze(0).cpu().detach()
img = img.permute(1, 2, 0)
fig.add_subplot(1, 2, 1)
plt.imshow(img, cmap="gray")

# showing predicted segmentation mask after doing some post processing
pred_mask = pred_mask.squeeze(0).cpu().detach()
pred_mask = pred_mask.permute(1, 2, 0)
pred_mask[pred_mask < 0]=0
pred_mask[pred_mask > 0]=1
fig.add_subplot(1, 2, 2)
plt.imshow(pred_mask, cmap="gray")

plt.show()